In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
dataset = pd.read_csv('../Datasets/Heart Disease/heart.csv')

In [3]:
X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# TRAIN MODEL

In [4]:
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=120)
automl.fit(X_train.values, y_train)
y_hat = automl.predict(X_test.values)

In [5]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.9922178988326849

In [8]:
import pickle
with open('./models/heart_disease_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

# LOAD MODEL

In [4]:
import pickle
with open('./models/heart_disease_automl.pkl', 'rb') as f:
    automl = pickle.load(f)

len(automl.show_models())

21

# LIME

In [11]:
import lime
import lime.lime_tabular
import tqdm 

In [16]:
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_features = X_train.columns.drop(continuous_features).tolist()
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=['No Disease', 'Disease'], categorical_features=categorical_features, discretize_continuous=True)

In [17]:
text_x = X_test.values

In [18]:
exp_fn = lambda i: explainer.explain_instance(X_test.iloc[i], automl.predict_proba, num_features=len(X_test.columns))
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [10]:
import metrics

In [20]:
exp1 = exp_fn_wrap(text_x)
exp2 = exp_fn_wrap(text_x)

100%|██████████| 257/257 [29:48<00:00,  6.96s/it]


In [23]:
with open('./explanations/heart_disease1.pkl', 'wb') as f:
    pickle.dump(exp1, f)
    
with open('./explanations/heart_disease2.pkl', 'wb') as f:
    pickle.dump(exp2, f)

In [12]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [30]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(X_test.columns))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [31]:
i, s, sb

((100.0, 0, 257), (0, 257, 66049, 0.0), (66, 257))

In [33]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [34]:
sim

0.29290093274366613

In [ ]:
trust = metrics.calc_trust_score(automl, text_x, exp1, 3, X_train.columns.tolist())

In [ ]:
trust

In [39]:
X_train.columns.tolist()

['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal']

# CIU

In [5]:
from ciu import determine_ciu
import tqdm

In [6]:
feat_list = X_train.columns.tolist()

In [7]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = determine_ciu(X_test.iloc[i:i+1], automl.predict_proba, X_train.to_dict('list'), samples = 1000, prediction_index = 1)
        exp_list = [[feat_list.index(i), exp.ci[i]] for i in exp.ci]
        exp1.append(exp_list)
    return np.array(exp1)

In [8]:
exp1 = exp_fn_blk(X_test)
exp2 = exp_fn_blk(X_test)

100%|██████████| 257/257 [17:59<00:00,  4.20s/it]


In [18]:
np.save('./explanations/heart_disease_ciu1.npy', exp1)
np.save('./explanations/heart_disease_ciu2.npy', exp2)

In [13]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(feat_list))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [14]:
i, s, sb

((100.0, 0, 257), (0, 257, 66049, 0.0), (54, 257))

In [26]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [27]:
sim

0.6221040012099591

In [30]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(automl, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 257/257 [37:30<00:00,  8.76s/it]


In [31]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

0.14328473083336896
0.0
0.0
0.15384615384615383
0.0
0.0


In [34]:
metrics.calc_trust_score(automl, X_test.to_numpy(), exp1, 3, X_train.columns.to_list())

100%|██████████| 257/257 [2:09:37<00:00, 30.26s/it]  


0.474059662775616